In [1]:
!pip install pandas sentence-transformers scikit-learn

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
!pip install datasets

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from datasets import Dataset

# Loading the dataset
data = pd.read_csv('/content/purchase_order.csv')

data['combined'] = data.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Initialize the SentenceTransformer model for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the text data from the dataset
data['embeddings'] = data['Creation Date'].apply(lambda x: model.encode(x))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
generation_args = {
    "max_new_tokens": 512,     # Maximum length of the response
    "return_full_text": False,      # Only return the generated text
}

In [8]:
def query(messages):
    """Sends a conversation history to the AI assistant and returns the answer.

    Args:
        messages (list): A list of dictionaries, each with "role" and "content" keys.

    Returns:
        str: The answer from the AI assistant.
    """

    # Extract the last user message
    user_message = messages[-1]['content']

    # Embed the user's message
    user_embedding = model.encode(user_message)

    # Compute similarities between the user message and the dataset embeddings
    similarities = cosine_similarity([user_embedding], list(data['embeddings'].values))

    # Find the index of the most similar entry in the dataset
    closest_index = np.argmax(similarities)

    # Retrieve the row that is the most similar and return all the information in that row
    closest_match = data.iloc[closest_index]

    return closest_match.to_string(index=False)

def chat():
    """Enables interactive chat sessions with the AI assistant."""

    # Initialize the conversation with instructions for the AI assistant
    conversation_history = [
        {"role": "system", "content": "You are a helpful digital assistant. Please provide accurate information to the user."}
    ]

    # Main chat loop
    while True:
        user_input = input("You: ")

        # Check if the user wants to exit the chat
        if user_input.lower() == "exit":
            break

        # Add user's message to the conversation history
        conversation_history.append({"role": "user", "content": user_input})

        # Get a response from the AI assistant
        try:
            response = query(conversation_history)
            print("Assistant: ", response)

            # Record the AI assistant's response in the conversation history
            conversation_history.append({"role": "assistant", "content": response})

        except Exception as e:
            print(f"An error occurred: {e}, please try again.")

# Start the chat
chat()

You: give me all the detail of a purchase made in 25 June
Assistant:                            66db1469e71037c2844635e5
                                         6/25/2015
                                         6/25/2015
                                         2014-2015
                                               NaN
                                          BMB43810
                                      NON-IT Goods
                              Informal Competitive
                  Fish and Wildlife, Department of
                                               0.0
                                           Unknown
                                                NO
                                   18 foot Pontoon
                                              boat
                                               1.0
                                       $20,282.00 
                                       $20,282.00 
                                          25111802
            